<a href="https://colab.research.google.com/github/vishal-burman/PyTorch-Architectures/blob/master/misc/T5Unjumble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install transformers
! pip install datasets

In [ ]:
! pip install sentencepiece

In [1]:
import torch
from torch.utils.data import DataLoader, Dataset

from transformers import T5ForConditionalGeneration, T5Tokenizer
from datasets import load_dataset

dataset = load_dataset('quora')

Using custom data configuration default
Reusing dataset quora (/root/.cache/huggingface/datasets/quora/default/0.0.0/2be517cf0ac6de94b77a103a36b141347a13f40637fbebaccb56ddbe397876be)


In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [12]:
question_list = dataset['train']['questions']

In [13]:
sent_train = [question['text'][0] for question in question_list[:10000]]
sent_valid = [question['text'][0] for question in question_list[10000:15000]]

In [36]:
class UJDataset(Dataset):
  def __init__(self, sent_list):
    self.sent_list = sent_list
    self.sample_list = []
    self.build()

  def __len__(self):
    return len(self.sample_list)
  
  def __getitem__(self, idx):
    sample = self.sample_list[idx]
    return {
        'src': sample['src'],
        'tgt': sample['tgt'],
    }
  
  def build(self):
    for sent in self.sent_list:
      unj = sent
      words = sent.split(" ")
      random.shuffle(words)
      jum = ' '.join(words)
      self.sample_list.append({'src': unj, 'tgt': jum})

In [41]:
train_dataset = UJDataset(sent_train)
valid_dataset = UJDataset(sent_valid)

In [42]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')
model.to(device)

params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('Total Trainable Parameters: ', params)

Some weights of the model checkpoint at t5-small were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Total Trainable Parameters:  60506624


In [43]:
BATCH_SIZE = 16
EPOCHS = 5
LEARNING_RATE = 3e-5
MAX_LENGTH=16

train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

In [ ]:
start_time = time.time()
for epoch in range(EPOCHS):
  model.train()
  for idx, sample in enumerate(train_loader):
    tokens = tokenizer.prepare_seq2seq_batch(src_texts=sample['src'], tgt_texts=sample['tgt'], max_length=MAX_LENGTH, max_target_length=MAX_LENGTH, padding='max_length', truncation=True)
    ids = tokens['input_ids'].to(device)
    mask = tokens['attention_mask'].to(device)
    tgt = tokens['labels'].to(device)
    outputs = model(input_ids=ids, attention_mask=mask, labels=tgt)
    